<div style="text-align:center">
<img src="img/nlopt2.gif" width="600">
</div>

# <b> Summary </b>
1. <b> Gemini AI : Répartition de la charge électrique (Economic Dispatch)</b>
    - Abstract
    - Instance
    - Math model
    - Cplex solution


# <b> Gemini : Répartition de la charge électrique (Economic Dispatch)</b>



<div style="text-align:center">
<img src="img/top-10-nuclear-power-companies.png" width="400">
</div>

## <b>Abstract</b>
L'optimisation quadratique (QP) est extrêmement courante dans le monde réel, notamment pour équilibrer les coûts et la performance. Un exemple parfait est la répartition de la charge électrique (Economic Dispatch).

Imaginons que vous dirigiez une petite centrale électrique avec deux générateurs. Votre but est de produire une quantité précise d'électricité au coût le plus bas possible.

Le Problème : Pourquoi "Quadratique" ?

Le coût de fonctionnement d'un générateur n'est pas linéaire. Plus on le sollicite, plus il perd en efficacité (chaleur, usure), ce qui fait grimper le coût de façon exponentielle. On modélise souvent cela par une fonction du second degré :

Cout=ax²+bx+c

## <b>Instance</b>


<div style="text-align:center">
<img src="img/Screenshot 2026-01-22 at 21-30-55 Google Gemini.png" width="400">
</div>

## <b>Math model</b>
Comin' soon
## <b>Cplex solution</b>
<div style="text-align:center">
<img src="img/IBM_CPLEX_0.png">
</div>

In [3]:
from docplex.mp.model import Model

# 1. Créer le modèle
mdl = Model(name='Optimisation_Energie')

# 2. Variables de décision (Puissance produite par chaque générateur en MW)
# On définit des bornes (ex: min 10MW, max 200MW)
x1 = mdl.continuous_var(lb=10, ub=200, name='gen1')
x2 = mdl.continuous_var(lb=10, ub=200, name='gen2')

# 3. Fonction Objectif : Minimiser le coût total (Quadratique)
# Coût = (0.5*x1^2 + 2*x1) + (0.25*x2^2 + 8*x2)
mdl.minimize(0.5*x1**2 + 2*x1 + 0.25*x2**2 + 8*x2)

# 4. Contrainte : Satisfaire la demande de 150 MW
mdl.add_constraint(x1 + x2 == 150, "Demande_Totale")

# 5. Résolution
solution = mdl.solve()

# 6. Affichage des résultats
if solution:
    print(f"Statut : {mdl.get_solve_status()}")
    print(f"Puissance Générateur 1 : {solution[x1]:.2f} MW")
    print(f"Puissance Générateur 2 : {solution[x2]:.2f} MW")
    print(f"Coût Total : {solution.objective_value:.2f} €")
else:
    print("Aucune solution trouvée.")

Statut : JobSolveStatus.OPTIMAL_SOLUTION
Puissance Générateur 1 : 54.00 MW
Puissance Générateur 2 : 96.00 MW
Coût Total : 4638.00 €


Pourquoi c'est intelligent ?

Si le coût était linéaire, CPLEX choisirait simplement le générateur le moins cher et le ferait tourner au maximum. Ici, comme le coût est quadratique, CPLEX calcule le point d'équilibre où l'augmentation du coût marginal du premier générateur devient plus élevée que celle du deuxième.

Dans cet exemple, CPLEX va répartir la charge de manière à ce que les "pentes" des coûts soient égales, minimisant ainsi la facture totale.

Ce que CPLEX apporte de plus

Dans un vrai scénario industriel, CPLEX gérerait en plus :

    Les pertes en ligne (la résistance des câbles).

    Les émissions de CO2 (une autre fonction quadratique à minimiser).

    Le démarrage/arrêt (variables binaires, ce qui transforme le problème en MIQP - Mixed Integer Quadratic Programming).